# <font color="AA0088">Exam Scores</font>

## <font color="AA0088">Project Participants</font>

Stakeholder(s): Me

Project Manager(s): Me

Data Analyst(s): erm.... Me

## <font color="AA0088">Problem Definition</font>

The purpose of this project is to gather insights from a collection of exam scores to understand the following main goals:

* The average score for each gender
* The average score for each group

## <font color="AA0088">Import Required Libraries</font>

The below libraries are required by this notebook.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from platform import python_version
from plotly.subplots import make_subplots

import plotly.graph_objs as go
from plotly import tools

Show the versions of the libraries that will be used.

In [2]:
print(f"Numpy Version:  {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Python Version: {python_version()}")

Numpy Version:  1.23.3
Pandas Version: 1.4.4
Python Version: 3.10.6


## <font color="AA0088">Data Processing</font>

### Data Source(s)

The data for this project was acquired from the following location(s):

### Data Overview

Give an overview of what the data represents as a whole.

### Data Dictionary

No data dictionary has been provided by the supplier of the data. 

From initial investigation of the features, a data dictionary has been created and can be found [here](data/raw-data-dictionary.xlsx).

**<font color="AA0088">NOTE</font>**: The contents of the description for each feature are assumptions but are logical assumptions based on the description of each feature.

### Import The Raw Data

First step is to import the data to a pandas dataframe from the source of the data. In this case, the source data will be a csv file.

In [3]:
raw_data_df = pd.read_csv("./data/raw_data.csv")

### Details About The Raw Data Dataframe

Let us have a quick look at the first five rows of the dataframe.

In [4]:
raw_data_df.head(n = 5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


Now let's have a look at the information about the raw_data_df.

In [5]:
raw_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


Initial observations are that there are:

* 8 features which are mostly:
  * string (object)
  * integers (int64)
* 1000 rows
* No null values but that will be checked soon

Some of the features have spaces or / in their names so they should be changed to _ to make querying them simpler.

### Cleaning Up The Data

Before performing any data cleaning, the names of each feature with a space or a / need to be changed to an _.

First, make a copy of the raw_data_df so that it stays in tact in case it is needed later on.

In [6]:
exam_scores_df = raw_data_df.copy()
exam_scores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


Now let's change the names of the features to remove the spaces and /'s. 

Once done, the columns will then have any uppercase values converted to lowercase.

In [7]:
exam_scores_df.columns = exam_scores_df.columns.str.replace(" ", "_")
exam_scores_df.columns = exam_scores_df.columns.str.replace("/", "_")

# --- Convert any uppercase characters to lowercase:
exam_scores_df.columns = exam_scores_df.columns.str.lower()

In [8]:
exam_scores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


Now that the column names are in a more preferred format, let's have a look to see if there are any null values.

In [9]:
exam_scores_df.isna().sum()

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64

Ok, there are no null values in any of the features.

### Export The Working Set Dataframe

Before going any further, let's make a backup of the current progress of the dataframe to a csv file.

In [10]:
exam_scores_df.to_csv(path_or_buf = "./data/exam_scores.csv", 
                      index = False)

Now let's move onto the exploratory data analysis phase.

## <font color="AA0088">Exploratory Data Analysis</font>

First, let's define some custom classes and functions to create the plots / charts that will be used.

### Required Custom Classes and Functions

In [16]:
def create_pi_chart(df_source:list, 
                    feature_name:str, 
                    fig_title:str):
    
    
    """_summary_
    This function will generate a pie plot / chart for a single feature that is passed to it as an argument.
    
    Args:
        df_source (list (Pandas DataFrame)): The name of the dataframe that you wish to use.
        feature_name (str): The name of the feature (column) to use from the dataframe.
        fig_title (str): The title that will be shown on the pie plot / chart.

    Returns:
        plotly graph (figure): The function will return a pie plot / chart depicting the data passed to this function.
    """
    
    
    fig_pie = px.pie(df_source,
                     values = df_source[feature_name].value_counts(),
                     names = df_source[feature_name].value_counts().index,
                     title = fig_title.title(),
                     width = 500,
                     color_discrete_sequence = px.colors.sequential.Agsunset,
                     )


    fig_pie.update_traces(textposition="inside", 
                          textinfo="label+percent",
                          textfont_size = 14,
                          textfont_color = "white",
                          hovertemplate = "<b>%{label}</b><br><br>Total: %{value}</br>Percent: %{percent}",
                          )
    
    
    fig_pie.update_layout({"title_font_size": 24,
                           "title_x": 0.50,})
    
    
    return fig_pie

### Total examinees by gender

In [17]:
fig_pie_gender = create_pi_chart(df_source = exam_scores_df, 
                                 feature_name = "gender", 
                                 fig_title = "Total examinees by gender")


fig_pie_gender.show();

### Total examinees by race / ethnicity

In [13]:
fig_pie_race = create_pi_chart(df_source = exam_scores_df, 
                               feature_name = "race_ethnicity", 
                               fig_title = "Total examinees by race / ethnicity")


fig_pie_race.show();

### Average Scores By Gender

### Average Scores By Group